In [4]:
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame
import sqlite3

from DATA225utils import make_connection, dataframe_query

In [5]:
#!pip install mysql-connector-python


In [6]:
conn = make_connection(config_file = 'oltp.ini')
cursor = conn.cursor()

In [8]:
main_df = pd.read_csv('trends.csv').dropna(axis=1, how = 'all')
#main_df.head(3)

In [9]:
main_df['Date_of_Purchase'] = pd.to_datetime(main_df['Date_of_Purchase'])

main_df['Date_of_Purchase'] = main_df['Date_of_Purchase'].apply(lambda x: x.date())


In [22]:
""" cursor.execute('''-- -- Drop Foreign Key Constraints
ALTER TABLE Payment DROP FOREIGN KEY fk_Payment_Purchases;
ALTER TABLE Shipping DROP FOREIGN KEY fk_Shipping_Purchases;
ALTER TABLE Location DROP FOREIGN KEY fk_Location_Purchases;
ALTER TABLE Purchases DROP FOREIGN KEY fk_Purchases_Customers;
ALTER TABLE Purchases DROP FOREIGN KEY fk_Purchases_Items;
ALTER TABLE Shipping DROP FOREIGN KEY fk_Shipping_Purchases;

-- Drop Tables
DROP TABLE IF EXISTS Customers;
DROP TABLE IF EXISTS Items;
DROP TABLE IF EXISTS ItemDetails;
DROP TABLE IF EXISTS Payment;
DROP TABLE IF EXISTS Location;
DROP TABLE IF EXISTS Shipping;
DROP TABLE IF EXISTS Purchases;

''' )

# Commit the table creations
conn.commit()"""

" cursor.execute('''-- -- Drop Foreign Key Constraints\nALTER TABLE Payment DROP FOREIGN KEY fk_Payment_Purchases;\nALTER TABLE Shipping DROP FOREIGN KEY fk_Shipping_Purchases;\nALTER TABLE Location DROP FOREIGN KEY fk_Location_Purchases;\nALTER TABLE Purchases DROP FOREIGN KEY fk_Purchases_Customers;\nALTER TABLE Purchases DROP FOREIGN KEY fk_Purchases_Items;\nALTER TABLE Shipping DROP FOREIGN KEY fk_Shipping_Purchases;\n\n-- Drop Tables\nDROP TABLE IF EXISTS Customers;\nDROP TABLE IF EXISTS Items;\nDROP TABLE IF EXISTS ItemDetails;\nDROP TABLE IF EXISTS Payment;\nDROP TABLE IF EXISTS Location;\nDROP TABLE IF EXISTS Shipping;\nDROP TABLE IF EXISTS Purchases;\n\n''' )\n\n# Commit the table creations\nconn.commit()"

In [10]:
cursor.execute('''

CREATE TABLE Customers
(
  Customer_ID INT NOT NULL,
  Age INT NOT NULL,
  Gender ENUM('Male', 'Female', 'Other') NOT NULL,
  PreviousPurchases INT NOT NULL,
  PRIMARY KEY (Customer_ID)
);

''')


cursor.execute('''
CREATE TABLE Items
(
  Item_ID INT NOT NULL,
  ItemName VARCHAR(255) NOT NULL,
  Category VARCHAR(255) NOT NULL,
  PRIMARY KEY (Item_ID)
);
''')

cursor.execute('''
CREATE TABLE ItemDetails
(
  ItemDetails_ID INT NOT NULL,
  Size VARCHAR(5) NOT NULL,
  Color VARCHAR(255) NOT NULL,
  Season VARCHAR(255) NOT NULL,
  Item_ID INT NOT NULL,
  PRIMARY KEY (ItemDetails_ID),
  FOREIGN KEY (Item_ID) REFERENCES Items(Item_ID) ON DELETE CASCADE
);
''' )


cursor.execute('''
               
CREATE TABLE Location
(
  Location_ID INT NOT NULL,
  Region VARCHAR(255) NOT NULL,
  State VARCHAR(255) NOT NULL,
  City VARCHAR(255) NOT NULL,
  PRIMARY KEY (Location_ID));
''')



cursor.execute('''

CREATE TABLE Payment
(
  Payment_ID INT NOT NULL,
  PaymentMethod VARCHAR(255) NOT NULL,
  PRIMARY KEY (Payment_ID) );
''')


cursor.execute('''
              
CREATE TABLE Shipping
(
  Shipping_ID INT NOT NULL,
  Shipping_type VARCHAR(255) NOT NULL,
  DiscountApplied VARCHAR(255),
  PromocodeUsed VARCHAR(255) NOT NULL,
  PRIMARY KEY (Shipping_ID),
  Payment_ID INT,
  FOREIGN KEY (Payment_ID) REFERENCES Payment(Payment_ID) ON DELETE CASCADE);
''')

cursor.execute('''

CREATE TABLE Purchases
(
  Purchase_ID INT NOT NULL,
  Purchase_Date DATE NOT NULL,
  Purchase_Amount DECIMAL(10, 2) NOT NULL,
  ReviewRating DECIMAL(3, 2) CHECK (ReviewRating BETWEEN 1.00 AND 5.00) NOT NULL,
  Purchase_frequency VARCHAR(255) NOT NULL,
  Customer_ID INT,
  Item_ID INT,
  Location_ID INT,
  Payment_ID INT,
  PRIMARY KEY (Purchase_ID),
  FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID) ON DELETE CASCADE,
  FOREIGN KEY (Item_ID) REFERENCES Items(Item_ID) ON DELETE CASCADE,
  FOREIGN KEY (Location_ID) REFERENCES Location(Location_ID) ON DELETE CASCADE,
  FOREIGN KEY (Payment_ID) REFERENCES Payment(Payment_ID) ON DELETE CASCADE );
  
''')





# Commit the table creations
conn.commit()

In [11]:
main_df.head(3)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Region,State,City,Size,...,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,Date_of_Purchase
0,17.0,25.0,Male,Sunglasses,Accessories,36.0,Southeast,Alabama,Birmingham,S,...,Spring,4.1,Yes,Next Day Air,Yes,Yes,44.0,Debit Card,Bi-Weekly,2022-10-11
1,65.0,65.0,Male,Boots,Footwear,83.0,Southeast,Alabama,Birmingham,S,...,Fall,4.8,Yes,Standard,Yes,Yes,18.0,Debit Card,Fortnightly,2023-08-09
2,71.0,22.0,Male,Belt,Accessories,29.0,Southeast,Alabama,Birmingham,M,...,Fall,4.2,Yes,Express,Yes,Yes,32.0,Debit Card,Every 3 Months,2023-06-06


## Customers 

In [12]:
customer_df = main_df[['Customer ID', 'Age', 'Gender','Previous Purchases']].dropna(subset=['Customer ID'])


for row in customer_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.Customers VALUES (%s,%s,%s,%s)", row)
conn.commit()


## Items

In [13]:

Products_df = main_df[['Item Purchased', 'Category']].drop_duplicates(subset=['Item Purchased', 'Category']).dropna()
Products_df = Products_df.sort_values(by=['Item Purchased','Category'])

Products_df['ItemID'] = range(1, len(Products_df) + 1)

Products_df= Products_df[['ItemID','Item Purchased','Category']]


for row in Products_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.Items VALUES (%s,%s,%s)", row)

conn.commit()



## Item Details

In [14]:
joined_df = main_df.merge(Products_df, how='left', on=['Item Purchased','Category'])

item_details_df = joined_df[['Size', 'Color','Season','ItemID']].drop_duplicates().dropna()

item_details_df['ItemDetails_ID'] = range(1, len(item_details_df) + 1)

item_details_df= item_details_df[['ItemDetails_ID','Size','Color', 'Season','ItemID']]

for row in item_details_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.ItemDetails VALUES (%s,%s,%s,%s,'%s')", row)

conn.commit()



## Locations

In [15]:
Locations_df = main_df[['Region', 'State','City']].drop_duplicates().dropna()

Locations_df = Locations_df.sort_values(by=['Region', 'State','City'])

Locations_df['LocationID'] = range(1, len(Locations_df) + 1)

Locations_df= Locations_df[['LocationID','Region', 'State','City']]

for row in Locations_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.Location VALUES (%s,%s,%s,%s)", row)

conn.commit()


## Payments

In [16]:
payment_df = main_df['Payment Method'].drop_duplicates().dropna().reset_index(drop=True)
payment_df = pd.DataFrame({'Payment_ID': range(1, len(payment_df) + 1), 'Payment Method': payment_df})


for row in payment_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.Payment VALUES (%s,%s)", row)

conn.commit()

In [17]:
payment_df

,Payment_ID,Payment Method
0,1,Debit Card
1,2,Bank Transfer
2,3,Cash
3,4,Credit Card
4,5,PayPal
5,6,Venmo


## Shipping

In [18]:
joined_df = main_df.merge(payment_df, how='left', on='Payment Method')

# Extract unique shipping information and assign unique IDs
shipping_df = joined_df[['Shipping Type', 'Discount Applied', 'Promo Code Used', 'Payment_ID']].drop_duplicates().dropna().reset_index(drop=True)
shipping_df['Shipping_ID'] = range(1, len(shipping_df) + 1)
shipping_df = shipping_df[['Shipping_ID', 'Shipping Type', 'Discount Applied', 'Promo Code Used', 'Payment_ID']]

# Display the new DataFrame
print(shipping_df)


for row in shipping_df.itertuples(index=False):
    # print(row)
    cursor.execute("INSERT INTO datadynamos_db.Shipping VALUES (%s, %s, %s, %s, %s)", row)

    
conn.commit()


    Shipping_ID   Shipping Type Discount Applied Promo Code Used  Payment_ID
0             1    Next Day Air              Yes             Yes         1.0
1             2        Standard              Yes             Yes         1.0
2             3         Express              Yes             Yes         1.0
3             4    Next Day Air              Yes             Yes         2.0
4             5    Store Pickup              Yes             Yes         1.0
..          ...             ...              ...             ...         ...
67           68  2-Day Shipping              Yes             Yes         3.0
68           69    Next Day Air              Yes             Yes         3.0
69           70   Free Shipping              Yes             Yes         2.0
70           71    Next Day Air              Yes             Yes         4.0
71           72        Standard              Yes             Yes         3.0

[72 rows x 5 columns]


## Purchases

In [19]:
joined_df = main_df.merge(customer_df, how='left', on='Customer ID') \
                .merge(Products_df, how='left', on=['Item Purchased','Category']) \
                .merge(Locations_df, how='left', on=['Region', 'State','City'] ) \
                .merge(payment_df, how='left', on=['Payment Method'])
                

purchase_df = joined_df[['Date_of_Purchase','Purchase Amount (USD)','Review Rating','Frequency of Purchases','Customer ID','ItemID','LocationID','Payment_ID']].drop_duplicates().dropna()


purchase_df['Purchase_ID'] = range(1, len(purchase_df) + 1)

purchase_df= purchase_df[['Purchase_ID','Date_of_Purchase','Purchase Amount (USD)','Review Rating','Frequency of Purchases','Customer ID','ItemID','LocationID','Payment_ID']]


for row in purchase_df.itertuples(index=False):
    #print(row)
    cursor.execute("INSERT INTO datadynamos_db.Purchases VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)", row)

conn.commit()




## 